In [3]:
pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 21.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import mysql.connector
import csv

In [1]:
def get_db_connection():
    connection = None
    try:
        connection = mysql.connector.connect(user='root',
                                             password='Daringscape3#',
                                             host='localhost',
                                             port='3306',
                                             database='DE_database')
    except Exception as error:
        print("Error while connecting to database for job tracker", error)
    return connection

In [3]:
class TicketSystem:
    def __init__(self, user, password, host, port, database):
        self.user = user
        self.password = password
        self.host = host
        self.port = port
        self.database = database
        self.connection = None

    def get_db_connection(self):
        try:
            self.connection = mysql.connector.connect(
                user=self.user,
                password=self.password,
                host=self.host,
                port=self.port,
                database=self.database
            )
        except Exception as error:
            print("Error while connecting to database:", error)
        return self.connection

In [4]:
class TicketSystem:
    # ...

    def load_third_party(self, file_path_csv):
        try:
            connection = self.get_db_connection()
            cursor = connection.cursor()

            with open(file_path_csv, 'r') as file:
                csv_data = csv.reader(file)
                next(csv_data)  # Skip header row
                for row in csv_data:
                    sql = "INSERT INTO sales (ticket_id, trans_date, event_id, event_name, event_date, event_type, event_city, customer_id, price, num_tickets) " \
                          "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
                    cursor.execute(sql, row)

            connection.commit()
            cursor.close()
            connection.close()
            print("Data loaded successfully.")
        except Exception as error:
            print("Error while loading data:", error)

In [5]:
from datetime import datetime, timedelta

class TicketSystem:
    # ...

    def query_popular_tickets(self):
        try:
            connection = self.get_db_connection()
            cursor = connection.cursor()

            # Get the date one month ago
            one_month_ago = datetime.now() - timedelta(days=30)
            sql = "SELECT event_name, SUM(num_tickets) as total_tickets_sold " \
                  "FROM sales " \
                  "WHERE date(trans_date) >= %s " \
                  "GROUP BY event_name " \
                  "ORDER BY total_tickets_sold DESC " \
                  "LIMIT 3"
            cursor.execute(sql, (one_month_ago,))
            records = cursor.fetchall()

            cursor.close()
            connection.close()
            return records
        except Exception as error:
            print("Error while querying data:", error)

In [6]:
class TicketSystem:
    # ...

    def display_popular_tickets(self):
        records = self.query_popular_tickets()
        if records:
            print("Here are the most popular tickets in the past month:")
            for record in records:
                print(f"- {record[0]}")
        else:
            print("No popular tickets found.")

In [7]:
ticket_system = TicketSystem(user='root', password='Daringscape3#', host='localhost', port='3306', database='DE_database')
ticket_system.load_third_party('third_party_sales_1.csv')
ticket_system.display_popular_tickets()

TypeError: TicketSystem() takes no arguments